In [1]:
import torch
from transformers import AutoModel, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel, PeftConfig

# 定义全局变量和参数
model_name_or_path = 'THUDM/chatglm3-6b'  # 模型ID或本地路径
peft_model_path = f"/root/autodl-tmp/models/THUDM/chatglm3-6b/20240330171809"

In [2]:
config = PeftConfig.from_pretrained(peft_model_path)

q_config = BitsAndBytesConfig(load_in_4bit=True,
                              bnb_4bit_quant_type='nf4',
                              bnb_4bit_use_double_quant=True,
                              bnb_4bit_compute_dtype=torch.float32)

base_model = AutoModel.from_pretrained(config.base_model_name_or_path,
                                       quantization_config=q_config,
                                       trust_remote_code=True,
                                       device_map='auto')
base_model.requires_grad_(False)
base_model.eval()

Request b666d422-2ed1-4e03-aa1d-a790b049595b: HEAD https://hf-mirror.com/THUDM/chatglm3-6b/resolve/main/config.json (authenticated: False)
Request 6e664fa0-bca3-4058-9476-71a14da5a4bb: HEAD https://hf-mirror.com/THUDM/chatglm3-6b/resolve/main/configuration_chatglm.py (authenticated: False)
Request 21cd02ea-d839-4985-b493-9dc6e9e932d9: HEAD https://hf-mirror.com/THUDM/chatglm3-6b/resolve/main/modeling_chatglm.py (authenticated: False)


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

Request 95aefbd7-118f-4665-ae18-f64d12cdde7e: HEAD https://hf-mirror.com/THUDM/chatglm3-6b/resolve/main/generation_config.json (authenticated: False)


ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (embedding): Embedding(
      (word_embeddings): Embedding(65024, 4096)
    )
    (rotary_pos_emb): RotaryEmbedding()
    (encoder): GLMTransformer(
      (layers): ModuleList(
        (0-27): 28 x GLMBlock(
          (input_layernorm): RMSNorm()
          (self_attention): SelfAttention(
            (query_key_value): Linear4bit(in_features=4096, out_features=4608, bias=True)
            (core_attention): CoreAttention(
              (attention_dropout): Dropout(p=0.0, inplace=False)
            )
            (dense): Linear4bit(in_features=4096, out_features=4096, bias=False)
          )
          (post_attention_layernorm): RMSNorm()
          (mlp): MLP(
            (dense_h_to_4h): Linear4bit(in_features=4096, out_features=27392, bias=False)
            (dense_4h_to_h): Linear4bit(in_features=13696, out_features=4096, bias=False)
          )
        )
      )
      (final_layernorm): RMSNorm()
    )
    (output_la

In [3]:
input_text = '类型#裙*版型#显瘦*风格#文艺*风格#简约*图案#印花*图案#撞色*裙下摆#压褶*裙长#连衣裙*裙领型#圆领'
print(f'输入：\n{input_text}')
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path, trust_remote_code=True)

Request f11c3817-6c40-46e3-8373-043c904d4a0f: HEAD https://hf-mirror.com/THUDM/chatglm3-6b/resolve/main/tokenizer_config.json (authenticated: False)


输入：
类型#裙*版型#显瘦*风格#文艺*风格#简约*图案#印花*图案#撞色*裙下摆#压褶*裙长#连衣裙*裙领型#圆领


Request 2149c85b-3c54-4cd7-8e7c-8437e2fbec09: HEAD https://hf-mirror.com/THUDM/chatglm3-6b/resolve/main/tokenization_chatglm.py (authenticated: False)
Setting eos_token is not supported, use the default one.
Setting pad_token is not supported, use the default one.
Setting unk_token is not supported, use the default one.


In [4]:
response, history = base_model.chat(tokenizer=tokenizer, query=input_text)
print(f'ChatGLM3-6B 微调前：\n{response}')

/root/miniconda3/envs/glm3/lib/python3.10/site-packages/bitsandbytes/nn/modules.py:391: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn('Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.')


ChatGLM3-6B 微调前：
根据您的描述，我为您推荐一款显瘦、简约文艺风格的印花连衣裙。这款裙子采用撞色设计，下摆为压褶，裙长适中，圆领设计显得优雅。穿上这款裙子，您会感受到撞色图案带来的活力与文艺风格的优雅气质，简约的设计和压褶的立体感会让您更加自信。这款连衣裙非常适合约会、聚会或出席正式场合，是一款不可错过的时尚单品。


In [5]:
model = PeftModel.from_pretrained(base_model, peft_model_path)
response, history = model.chat(tokenizer=tokenizer, query=input_text)
print(f'ChatGLM3-6B 微调后: \n{response}')


ChatGLM3-6B 微调后: 
这款连衣裙采用简约的圆领设计，修饰脸型，穿着舒适不紧勒，凸显优雅气质。裙身采用撞色印花设计，展现时尚感，彰显文艺气息。简约的版型，穿着显瘦，搭配压褶的设计，更显细节。
